### Competition URL

https://www.kaggle.com/c/whale-categorization-playground

In [ ]:
#importing libraries

import tensorflow as tf
import pandas as pd
import numpy as np
import os
from PIL import Image
from scipy.misc import imread, imsave, imresize
import heapq

In [ ]:
#converting all to RGB

img_list = []
img_names = os.listdir("train/train")
for img_name in (os.listdir("train/train")):
    img_list.append("train/train/" + img_name)
    
i = 0
for img in img_list:
    img = Image.open(img)
    rgbimg = Image.new("RGB", img.size)
    rgbimg.paste(img)
    rgbimg.save('new_train/{}'.format(img_names[i]))
    i += 1    
    
img_list = []
img_names = os.listdir("test/test")
for img_name in (os.listdir("test/test")):
    img_list.append("test/test/" + img_name)
    
i = 0
for img in img_list:
    img = Image.open(img)
    rgbimg = Image.new("RGB", img.size)
    rgbimg.paste(img)
    rgbimg.save('new_test/{}'.format(img_names[i]))
    i += 1    

In [ ]:
#getting output labels

def get_labels(predictions):
    
    input_train_file = "train.csv"
    train_dataframe = pd.read_csv(input_train_file, header = 0)
    train_labels = train_dataframe["Id"].as_matrix()

    labels_dict = {}
    counter = 0
    for i in range(9850):
        if train_labels[i] in labels_dict:
            labels_dict[train_labels[i]][1] += 1
        else:
            labels_dict[train_labels[i]] = [counter, 1]
            counter += 1
            
    largest = heapq.nlargest(5, predictions) 
    
    largest_indices = []
    
    for i in range(5):
        largest_indices.append(list(predictions).index(largest[i]))
        
    true_labels = []
    
    for i in range(5):
        for label in labels_dict:
            if labels_dict[label][0] == largest_indices[i]:
                true_labels.append(label)
                
    return true_labels            

In [ ]:
#getting train data

def get_data():
    
    input_train_file = "train.csv"
    train_dataframe = pd.read_csv(input_train_file, header = 0)
    train_labels = train_dataframe["Id"].as_matrix()

    labels_dict = {}
    counter = 0
    for i in range(9850):
        if train_labels[i] in labels_dict:
            labels_dict[train_labels[i]][1] += 1
        else:
            labels_dict[train_labels[i]] = [counter, 1]
            counter += 1

    img_list = []
    for img_name in (os.listdir("new_train")):
        img_list.append("new_train/" + img_name)

    indices_list = []
    for j in range(9850):
        indices_list.append(labels_dict[train_labels[j]][0])

    one_hot_labels = tf.one_hot(indices_list, 4251) 

    with tf.Session() as sess:
        one_hot_labels = one_hot_labels.eval()
    
    def _parse_function(filename, label):
        image_string = tf.read_file(filename)
        image_decoded = tf.image.decode_jpeg(image_string)
        image_resized = tf.image.resize_images(image_decoded, [512, 512])
        return image_resized, label

    filenames = tf.constant(img_list)

    labels = tf.constant(indices_list)

    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

    dataset = dataset.map(_parse_function)
    
    return dataset, one_hot_labels

In [ ]:
#getting test data

def get_test_data():
    
    img_list = []
    for img_name in (os.listdir("new_test")):
        img_list.append("new_test/" + img_name)
    
    def _parse_function(filename):
        image_string = tf.read_file(filename)
        image_decoded = tf.image.decode_jpeg(image_string)
        image_resized = tf.image.resize_images(image_decoded, [512, 512])
        return image_resized
    
    filenames = tf.constant(img_list)
    
    dataset = tf.data.Dataset.from_tensor_slices((filenames))

    dataset = dataset.map(_parse_function)
    
    return dataset

In [ ]:
#creating the model 

def model():
    
    features = tf.placeholder(tf.float32, shape = (50, 512, 512, 3), name = "features")
    labels = tf.placeholder(tf.float32, shape = (50, 4251), name = "labels")
    
    input_layer = tf.reshape(features, (50, 512, 512, 3))
    
    conv1 = tf.contrib.layers.conv2d(
    inputs = input_layer,
    num_outputs = 16,
    kernel_size = [3 , 3],
    stride = 1,
    padding = 'SAME')
    
    pool1 = tf.contrib.layers.max_pool2d(
    inputs = conv1,
    kernel_size = [3 , 3],
    stride = 1,
    padding = 'VALID')
    
    conv2 = tf.contrib.layers.conv2d(
    inputs = pool1,
    num_outputs = 32,
    kernel_size = [5 , 5],
    stride = 3,
    padding = 'SAME')
    
    pool2 = tf.contrib.layers.max_pool2d(
    inputs = conv2,
    kernel_size = [3 , 3],
    stride = 1,
    padding = 'VALID')
    
    conv3 = tf.contrib.layers.conv2d(
    inputs = pool2,
    num_outputs = 64,
    kernel_size = [7 , 7],
    stride = 5,
    padding = 'SAME')
    
    pool3 = tf.contrib.layers.max_pool2d(
    inputs = conv3,
    kernel_size = [3 , 3],
    stride = 1,
    padding = 'VALID')
    
    conv4 = tf.contrib.layers.conv2d(
    inputs = pool3,
    num_outputs = 128,
    kernel_size = [9 , 9],
    stride = 7,
    padding = 'SAME')
    
    pool4 = tf.contrib.layers.max_pool2d(
    inputs = conv4,
    kernel_size = [3 , 3],
    stride = 1,
    padding = 'VALID')
    
    flatten_layer = tf.contrib.layers.flatten(pool4)
    
    fc1 = tf.contrib.layers.fully_connected(flatten_layer, 10000)
    fc2 = tf.contrib.layers.dropout(fc1, keep_prob = 0.8)
    fc3 = tf.contrib.layers.fully_connected(fc2, 6000)
    pred = tf.contrib.layers.fully_connected(fc3, 4251)
    
    loss = tf.losses.softmax_cross_entropy(labels, pred)
    
    return features, labels, pred, loss

In [ ]:
#running the model

def run_model(num_epochs):
    
    test_img_list =  os.listdir("new_test")
    
    dataset1, one_hot_labels = get_data()
    
    features, labels, pred, loss = model()
    
    dataset1 = dataset1.repeat().batch(50)
    
    iter = dataset1.make_one_shot_iterator()
    
    x, y = iter.get_next()
    
    dataset2 = get_test_data()
    
    dataset2 = dataset2.repeat().batch(50)
    
    iter = dataset2.make_one_shot_iterator()
    
    x_test = iter.get_next()
         
    optimizer = tf.train.MomentumOptimizer(0.001, 0.9).minimize(loss)
    
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        
        sess.run(init)
        for i in range(num_epochs):
            for j in range(197):
                new_x =x.eval()
                feed_dict = {features: new_x, labels: one_hot_labels[j * 50 : (j+1) * 50][:]}
                _, loss_value = sess.run([optimizer, loss], feed_dict)
                print("Epoch: {}, Iter: {}, Loss: {:.4f}".format(i, j, loss_value))
        
        save_path = saver.save(sess, "model/hbw_model.ckpt")
        
        print("Model saved in path: %s" % save_path)
        
        df = pd.DataFrame([], columns=['Image', ' Id'])
        m = 0
        
        for j in range(313):
            new_x_test = x_test.eval()
            feed_dict_test = {features: new_x_test}
            prediction = sess.run(pred, feed_dict)
            for k in range(len(prediction)):
                labels = get_labels(prediction[k])
                if m < len(test_img_list):
                    img_name = test_img_list[m]
                else:
                    break
                df = df.append({'Image': img_name, 'Id': labels[0] + " " + labels[1] + " " + labels[2] + " " + labels[3] + " " + labels[4]}, ignore_index=True)
                m += 1
        
        df.to_csv('outputs.csv', encoding='utf-8', index=False)

In [ ]:
run_model(100)